<a href="https://colab.research.google.com/github/hereagain-Y/TCR_VAE/blob/main/Model_speed_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Test latent distance calculation on GPU

In [1]:
! pip install Biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 4.8 MB/s 


In [2]:
import itertools
import pandas as pd 
import random
import time
from datetime import datetime
from pkgutil import extend_path
import numpy as np
import matplotlib.pyplot as plt

from six.moves import xrange
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np

from tqdm import tqdm
from torchvision.utils import save_image, make_grid
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
#
from scipy.spatial import distance
import random 
from Bio import pairwise2
from Bio.Align import substitution_matrices
from scipy import spatial
from scipy import stats

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
AAs= ['A', 'R', 'N', 'D', 'C', 'E', 'Q', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
index_code = {}
code_index = {}
l_max = 20
for i in range(len(AAs)):
    index_code[i] = AAs[i]
    code_index[ AAs[i] ] = i

def oneHotEncode(seq, l_max=l_max, index_code=index_code, code_index=code_index):
    n_amino = 20
    matrix = np.zeros((l_max,n_amino)).astype(int)
    for i in range(len(seq)):
        matrix[ i , code_index[seq[i]] ] = 1
    return matrix

# pca encoded 
pca_index = pd.read_csv("/content/drive/My Drive/DL/VAE/AA_indexPCA.csv")
d=pca_index.set_index('Unnamed: 0').T.to_dict('list')


# pca normalization
data = d.items()
list_dat = list(d.values())
arr = np.array(list_dat)
ex = np.array(arr)
ex_norm = (ex-ex.min(axis=0))/(ex.max(axis=0)-ex.min(axis=0))

AAs=np.array(list(d.keys()))
new_pca = {}

for i in np.arange(20):
    new_pca[AAs[i]]=ex_norm[i]

new_pca
d= new_pca

def AAindexEncoding(Seq):
    length_seq=len(Seq)
    global l_max
    AAE=np.zeros([l_max,20])
    if length_seq<l_max:
        for amino in range(length_seq):
            AA=Seq[amino]# 
            AAE[amino,]=d[AA] # add PC value 
            
        for amino in range(length_seq,l_max):
            AAE[amino,]=np.zeros(20)
    else: 
        for amino in range(length_seq): # zero padding
            AA=Seq[amino]# 
            AAE[amino,]=d[AA]
        
    #AAE=np.transpose(AAE.astype(np.float32)) # row as PC. and column as AA sequence 
    return AAE 

  
def GetFeatures(file):
    hot_encode=[]
    for seq in file:
        hot_encode.append(AAindexEncoding(seq))
    hot_encode=np.array(hot_encode)
    result=np.array(hot_encode)
    return(result)

In [5]:
matrix = substitution_matrices.load('BLOSUM62')
#from Bio.SubsMat import MatrixInfo as matlist
#matrix = matlist.blosum62
#in 80 max penalty for substitution is -6,
open_penalty = -4
gap_penalty = -4
#
def parseMatrix(m):
    re = {}
    alpha = m.alphabet
    for i in range(len(alpha)):
        for j in range(i,len(alpha)):
            re[(alpha[i],alpha[j])] = m[i,j]
    return re
align_matrix =  parseMatrix(substitution_matrices.load('BLOSUM62'))

In [16]:
seq_test = pd.read_csv('/content/drive/My Drive/DL/VAE/NormalCDR3.txt',delimiter='\t',header=None,names=['seq'])
seq_test['length'] = [len(seq) for seq in seq_test['seq']]

seq_test = seq_test[ seq_test['length']<=20 ]
seq = list( seq_test['seq'] )
seq_x = random.sample(seq,100)
input1 = []
input2 =[]
for pair in itertools.combinations(seq_x, 2):
  
    input1.append(pair[0])
    input2.append(pair[1])

# alignment on x y 
score_align=[]
for i in range(len(input1)):
    
  alignments = pairwise2.align.localds(input1[i], input2[i], align_matrix, open=open_penalty, extend=gap_penalty)
  score_align.append(alignments[0].score)

AA_matx= GetFeatures(input1)
AA_maty= GetFeatures(input2)


#seq_train_x = AA_matx # 20*20
#seq_train_y = AA_maty

#train_data=[]
#for i in range(len(AA_matx)):
#  train_data.append([AA_matx[i],score_align[i]])
trainloader_x = torch.utils.data.DataLoader([ [torch.from_numpy(AA_matx[i]).float(),score_align[i]] for i in range(len(score_align))],  batch_size=1000)
#i1, l1 = next(iter(trainloader))
#print(l1.shape)
trainloader_y = torch.utils.data.DataLoader([ [torch.from_numpy(AA_maty[i]).float(),score_align[i]] for i in range(len(score_align))],  batch_size=1000)

In [25]:
print(len(trainloader_x))

5


In [7]:
start_time=datetime.now()

In [22]:
cuda = True
channels =1
device = torch.device("cuda" if cuda else "cpu")


In [18]:

class VAE(nn.Module):
    def __init__(self,h_dim=64*10*10, z_dim=32):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            
            nn.Conv2d(1, 16, kernel_size=5, stride=1), #16
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=5, stride=1), # 12
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1), #10
            nn.ReLU()
        
        )

        
        
        # mean 64*5*5 =
        self.fc1 = nn.Linear(h_dim, z_dim)
        # var 
        self.fc2 = nn.Linear(h_dim, z_dim)
        # for decoder layer 
        self.fc3 = nn.Linear(z_dim, h_dim)
        
        self.decoder = nn.Sequential(
            
   
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, kernel_size=5, stride=1),
            nn.Sigmoid()
        )
        
    def reparameterize(self, mu):
        #std = logvar.mul(0.5).exp_()
        # return torch.normal(mu, std)
        #esp = torch.randn(*mu.size())
        z = mu
        return z
    
    def bottleneck(self, h):
        mu= self.fc1(h)
        z = self.reparameterize(mu)
        return mu, z
        
    def representation(self, x):
        return self.bottleneck(self.encoder(x))[0] # latent layer 

        
    
    #def alignmentscore(self,x,y):
    #    scores = pairwise2.align.localds(x,y,align_matrix,open=open_penalty,extend=gap_penalty)
    #    score_align = scores[0].score
    #    return score_align
        
    def latent(self,x,y):
        l1 = self.encoder(x)# 32
        l1 = l1.view(-1,64*10*10) # 32
        l1, mu1 = self.bottleneck(l1)

        l2 = self.encoder(y)# 32
        l2 = l2.view(-1,64*10*10) # 32
        l2, mu2 = self.bottleneck(l2)
        # use cos-simialrity
        #latent_dist =np.sqrt( np.sum(np.square(l1 - l2)) )
        
        
        return l1,l2

    def forward(self, x,y):
      # for dataset 1
        h1= self.encoder(x)
        h1 = h1.view(-1,64*10*10)
        z1, mu1 = self.bottleneck(h1) #
        z1 = self.fc3(z1) # 64*10*10
        z1 = z1.view(-1,64,10,10)
        z1 = self.decoder(z1)
      # for dat2   
        h2= self.encoder(y)
        h2 = h2.view(-1,64*10*10)
        z2, mu2 = self.bottleneck(h2)
        z2 = self.fc3(z2)
        z2 = z2.view(-1,64,10,10)
        z2 = self.decoder(z2)
        
        # aligment 
        #s = self.alignmentscore(x,y)
        # latent distance
        l1,l2 = self.latent(x,y)
        
        
        
        return z1, mu1,z2,mu2,l1,l2

In [19]:
model=torch.load('/content/drive/MyDrive/DL/CNNVAE/Paired_sigmoid_modified_1000_echo_cat_train.apx')

In [23]:
# move model to gpu
if torch.cuda.is_available():
    model.cuda()

In [20]:
distance1=[]
distance2=[]
delta=[]

In [24]:
for batch_idx, data in enumerate(zip(trainloader_x,trainloader_y)):
  x= data[0][0]
  y= data[1][0]
  score = data[0][1].to(device)
  x = x.view(len(x), 1,20,20)
  y = y.view(len(x), 1,20,20)
  x = x.to(device)
  y = y.to(device)
  x_hat, mean_x,y_hat,mean_y,l1,l2= model(x,y)
  dis = torch.nn.functional.pairwise_distance(l1, l2,2)
  delta.append(dis)
 

In [25]:
len(delta)

5

In [26]:
stacked_tensor = torch.cat(delta)
stacked_tensor.numel()

4950

In [27]:
distance=stacked_tensor.cpu().detach().numpy()

In [29]:
distance.shape

(4950,)

In [28]:
df_out = pd.DataFrame(columns=['seq1','seq2','alignment_score','latent_dist'])

In [30]:
df_out['seq1'] = input1
df_out['seq_2']=input2

df_out['alignment_score'] = score_align
df_out['latent_dist'] = distance
df_out.to_csv('/content/drive/MyDrive/DL/CNNVAE/speed_testing.csv',index=False)

